# Stage 1 - Scraping Lift Off!

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import os
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\willt\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
mars= {}

## NASA Mars News

In [4]:
# Source Scrape URL 
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)
time.sleep(3)

In [5]:
# Retrieve page
news_retrieve = requests.get(news_url)

# Create BeautifulSoup object
news_soup = BeautifulSoup(news_retrieve.text, 'html.parser')

news_html = browser.html
news_results = BeautifulSoup(news_html, 'html.parser')

In [6]:
# Retrieve headline and flavor text
news_title = news_results.find('div', class_='content_title').get_text()
news_teaser = news_results.find('div', class_='article_teaser_body').get_text()

print(news_title)
print(news_teaser)

news_dict = {'news_title': news_title, 'news_summary': news_teaser}

mars.update(news_dict)

NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network
Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.


## JPL Mars Space Images - Featured Image

In [7]:
# Source Scrape URL 
images_url = 'https://spaceimages-mars.com/'
browser.visit(images_url)
time.sleep(3)

In [8]:
# Retrieve page
images_retrieve = requests.get(images_url)

# Create BeautifulSoup object
images_soup = BeautifulSoup(images_retrieve.text, 'html.parser')

images_html = browser.html
images_results = BeautifulSoup(images_html, 'html.parser')

In [9]:
# Retrieve Current Header Image from URL 
featured_image = images_results.find('img', class_='headerimage')
featured_image = featured_image['src']
featured_image

'image/featured/mars1.jpg'

In [10]:
#Combine URl and header image
featured_image_url = images_url + featured_image
print(featured_image_url)
featured_image_url = {'featured_image': featured_image_url}
mars.update(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


## Mars Facts

In [11]:
# Source Scrape URL 
facts_url = 'https://galaxyfacts-mars.com/'

In [12]:
#Set table data into Dataframe
tables = pd.read_html(facts_url)
facts_df = tables[0]
facts_df.columns = ["Description","Mars_Data","Earth_Data"]
facts_df.set_index("Description", inplace=True)
facts_df

,Mars_Data,Earth_Data
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [13]:
#Converted DataFrame into HTML string
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars_Data</th>
      <th>Earth_Data</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [14]:
clean_html = facts_html.replace('\n', '')
clean_html
facts_dict={'mars_facts': clean_html}
mars.update(facts_dict)

In [15]:
# Save file as html 
mars_html = facts_df.to_html('mars_table.html')
mars_html

## Mars Hemishperes 

In [16]:
# URL of page to be scraped
hemis_url = 'https://marshemispheres.com/'
browser.visit(hemis_url)
time.(3)

hemis_html = browser.html
hemis_results = BeautifulSoup(hemis_html, 'html.parser')

In [17]:
# Get URL of pages to be scraped
url_results = hemis_results.find_all('div', class_='item')

url_list=[]

for result in url_results:
    
    hemisphere=result.find('a')['href']
    url_list.append(hemisphere)


print(url_list)

['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']


In [18]:
# Loop through URLs to scrape and get list of dictionaries
hemisphere_image_urls=[]

for item in url_list:
    item_url=hemis_url+item
    browser.visit(item_url)
    item_html=browser.html
    item_results=BeautifulSoup(item_html, 'html.parser')
    
    parent1=item_results.find('div', class_='cover')
    
    hemi_title=parent1.find('h2','title')
    hemi_title=hemi_title.text.strip()
    hemi_title=hemi_title.rstrip('Enhanced')
    hemi_title=hemi_title.rstrip(' ')
    
    parent2 = item_results.find('div', class_='downloads')
    first_image=parent2.find('ul')
    hemi_image_item = first_image.find_all('li')[0]
    hemi_image = hemi_image_item.find('a')['href']
    
    hemisphere_image_url={"title": hemi_title, "image_url": hemis_url+hemi_image}
    hemisphere_image_urls.append(hemisphere_image_url)

In [19]:
#Update mars dictionary
mars.update({'hemispheres':hemisphere_image_urls})

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'image_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [20]:
print(mars)

{'news_title': 'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network', 'news_summary': 'Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.', 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg', 'mars_facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars_Data</th>      <th>Earth_Data</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Mars - Earth Comparison</th>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    

In [21]:
browser.quit()

# Store Results in Mongo 

In [22]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [23]:
# Define database and collection
db = client.mars_db
collection = db.mars_info

In [24]:
collection.insert_one(mars)